In [1]:
#Library imports
import re
import os
import sys
import shutil
import time
import math
import gzip
import fnmatch
import random
import warnings
import numpy as np
import pandas as pd
import scipy.stats as scs
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict

import scipy.stats as scs
from sklearn.neighbors import BallTree, KDTree

import fiona
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from pyproj import Proj
import geoplot as gplt
import geoplot.crs as gcrs

# Allows access to scripts and modules relative to the parent directory.
parent = os.getcwd()
sys.path.append(os.path.join(parent, "functions"))

# Project specific user driven functions
from cleaning_functions import *

# My open source reusable user driven function repository.
from random_lumberjacks.src.random_lumberjacks.cleaning.cleaning_functions import *
from random_lumberjacks.src.random_lumberjacks.model.model_classes import *
from random_lumberjacks.src.random_lumberjacks.visualization.visualization_functions import *
from random_lumberjacks.src.random_lumberjacks.parsing.parse_noaa import *

#Notebook arguments
%matplotlib inline

In [2]:
station_key = pd.read_csv("data/noaa/isd-history.csv")

In [3]:
all_states = station_key[(station_key["CTRY"]=="US")]["STATE"].dropna().unique()
airshed = ['DE','IN','KY','MD','MI','MI','NC','NJ','NY','OH','PA','SC','TN','VA','VT','WV']
not_airshed = np.setdiff1d(all_states, airshed)

In [4]:
#Converting the end date to datetime to be able to select relevant years.
station_key["END"] = pd.to_datetime(station_key["END"], format="%Y%m%d")

#All data must have coordinates, be within the US, not be explicitly outside of the airshed, and be from 2001 or later.
relevant_stations = station_key[(station_key["CTRY"]=="US") & (station_key["STATE"].isin(not_airshed) == False) &
                                (station_key["END"] > "2001") & (station_key["LAT"].isna()==False)].reset_index().drop(columns = "index")


In [5]:
relevant_stations

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,621010,99999,MOORED BUOY,US,NaN,NaN,50.600,-2.933,-999.0,20080721,2008-07-21
1,621110,99999,MOORED BUOY,US,NaN,NaN,58.900,-0.200,-999.0,20041118,2004-11-18
2,621130,99999,MOORED BUOY,US,NaN,NaN,58.400,0.300,-999.0,20040726,2004-07-26
3,621160,99999,MOORED BUOY,US,NaN,NaN,58.100,1.800,-999.0,20040829,2004-08-29
4,621170,99999,MOORED BUOY,US,NaN,NaN,57.900,0.100,-999.0,20040726,2004-07-26
...,...,...,...,...,...,...,...,...,...,...,...
1533,A06773,334,TUCKER GUTHRIE MEMORIAL AIRPORT,US,KY,KI35,36.859,-83.358,473.1,20140731,2020-08-24
1534,A06800,120,TAZEWELL COUNTY AIRPORT,US,VA,KJFZ,37.067,-81.800,808.6,20140731,2020-08-24
1535,A06884,416,LURAY CAVERNS AIRPORT,US,VA,KLUA,38.667,-78.501,275.2,20140731,2020-08-24
1536,A07086,468,CARL R KELLER FIELD AIRPORT,US,OH,KPCW,41.516,-82.869,179.8,20140731,2020-08-24


In [14]:
cbp_cmc = pd.read_pickle("data/cbp_cmc.pickle")

In [15]:
locations_a = cbp_cmc.groupby(["Station"]).first().reset_index()[["Station", "Latitude", "Longitude"]]
locations_b = relevant_stations[["USAF", "LAT", "LON"]]
location_key = locations_a.copy()

In [8]:
tree = BallTree(np.deg2rad(locations_b[["LAT", "LON"]].values), metric='haversine')

In [9]:
distances, indices = tree.query(np.deg2rad(locations_a[["Latitude", "Longitude"]]), k = 3)
indices = pd.DataFrame(indices, columns = [f"id{i}" for i in np.arange(1,4)])
distances = pd.DataFrame(distances*3959, columns = [f"noaa_dist_mi{i}" for i in np.arange(1,4)])

In [16]:
for i, column in enumerate(indices.columns):
    location_key["USAF_"+column] = indices[column].map(lambda x: relevant_stations["USAF"][x])
    location_key["WBAN_"+column] = indices[column].map(lambda x: relevant_stations["WBAN"][x])
    location_key[f"noaa_dist_mi{i+1}"] = distances[f"noaa_dist_mi{i+1}"]

In [39]:
stations_to_query = pd.Index([])
for i in np.arange(1,4):
    group = location_key[f"USAF_id{i}"] + "-" + location_key[f"WBAN_id{i}"].map(lambda x: f"{x:05d}")
    stations_to_query = stations_to_query.union(group)

In [40]:
stations_to_query.union(group)

Index(['691174-99999', '691174-99999', '691174-99999', '691174-99999',
       '691174-99999', '691174-99999', '720285-03734', '720285-03734',
       '720285-03734', '720285-03734',
       ...
       '999999-64758', '999999-64758', '999999-64758', '999999-64758',
       '999999-64758', 'A00031-03725', 'A06884-00416', 'A06884-00416',
       'A06884-00416', 'A06884-00416'],
      dtype='object', length=2803)

In [129]:
relevant_stations[relevant_stations["USAF"]=="999999"]

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
1481,999999,178,LINDEN AIRPORT,US,NJ,KLDJ,40.617,-74.250,7.0,20140731,2016-12-31
1482,999999,370,PINEY ISLAND,US,NC,NaN,35.020,-76.460,5.2,20140731,2020-08-24
1483,999999,421,MADISONVILLE MUNICIPAL AIRPORT,US,KY,NaN,37.350,-87.400,124.4,20140731,2020-07-31
1484,999999,425,MIDDLEBURY STATE AIRPORT,US,VT,NaN,43.985,-73.095,149.1,20140731,2020-08-11
1485,999999,458,CLERMONT COUNTY AIRPORT,US,OH,NaN,39.078,-84.210,257.0,20140731,2020-08-24
1486,999999,3724,CARROLL CO RGNL/JJACK B POAGE FIELD AIRPORT,US,MI,KDMW,39.608,-77.008,240.5,20040102,2007-07-13
1487,999999,3728,MCCLELLANVILLE 7 NE,US,SC,NaN,33.153,-79.364,2.7,20020815,2020-08-24
1488,999999,3733,ELKINS 21 ENE,US,WV,NaN,39.013,-79.474,1033.3,20031117,2020-08-24
1489,999999,3739,CAPE CHARLES 5 ENE,US,VA,NaN,37.291,-75.927,8.8,20040303,2020-08-24
1490,999999,3755,GARRETT COUNTY AIRPORT,US,MD,NaN,39.580,-79.339,894.0,20061018,2020-08-01


,noaa_dist_mi1,noaa_dist_mi2,noaa_dist_mi3
0,4.685106,19.017453,19.522286
1,16.007895,18.902837,18.902837
2,31.604464,43.050463,53.025216
3,14.925637,35.984370,45.923375
4,7.260367,19.795753,31.204374
...,...,...,...
2474,0.446566,7.583171,7.790809
2475,2.192645,2.400609,5.253354
2476,1.638530,1.884531,7.337363
2477,3.573859,3.932400,4.018599


In [77]:
station_codes = relevant_stations["USAF"]

In [89]:
locations_b

,USAF,LAT,LON
13136,621010,50.600,-2.933
13138,621110,58.900,-0.200
13139,621130,58.400,0.300
13140,621160,58.100,1.800
13141,621170,57.900,0.100
...,...,...,...
29745,A06773,36.859,-83.358
29746,A06800,37.067,-81.800
29748,A06884,38.667,-78.501
29752,A07086,41.516,-82.869


In [83]:
locales

,Station,Date,Agency,CloudCover,Cruise,Database,FieldActivityEventType,FlowStage,GaugeHeight,GroupCode,...,Weather Conditions Yesterday,WindDirection,WindSpeed,Point,HUC12_,HUCNAME_,FIPS_,WATER TEMPERATURE DEG,SPECIFIC CONDUCTIVITY,AIR TEMPERATURE DEG
0,01491000,2011-10-12 09:00:00,MDDNR,NaN,NTN012,CBP,R,NaN,3.11,NaN,...,NaN,NaN,NaN,POINT (-75.78610999999999 38.99722),20600050203,Gravelly Branch-Choptank River,24011,17.400,NaN,NaN
1,01493112,2011-10-12 10:45:00,MDDNR,NaN,NTN012,CBP,R,NaN,1.31,NaN,...,NaN,NaN,NaN,POINT (-75.94014 39.25706),20600020406,Upper Chester River,24029,15.900,NaN,NaN
2,01502500,2005-10-08 12:05:00,SRBC,NaN,BAY428,CBP,R,Rising Flow (Routine),4.71,NaN,...,NaN,NaN,NaN,POINT (-75.40639 42.37778),20501010910,Lower Unadilla River,36017,13.900,NaN,NaN
3,01503000,2005-10-08 14:15:00,SRBC,NaN,BAY428,CBP,R,Rising Flow (Routine),2.75,NaN,...,NaN,NaN,NaN,POINT (-75.80333 42.03528),20501011312,Carlin Creek,36007,14.600,NaN,NaN
4,01511500,2012-01-19 14:00:00,SRBC,NaN,NTN012,CBP,R,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (-75.90916999999997 42.29806),20501020408,Lower Tioughnioga River,36007,0.150,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,YRK023.40,2005-08-02 07:58:00,VADEQ,Clear (0-10%),BAY424,CBP,NaN,NaN,NaN,NaN,...,NaN,WNW,1 To 10 Knots,POINT (-76.70888000000002 37.41742),20801070104,Skimino Creek-York River,51095,28.160,NaN,NaN
2475,YRK028.58,2005-08-02 10:33:00,VADEQ,Scattered To Partly Cloudy (10-50%),BAY424,CBP,NaN,NaN,NaN,NaN,...,NaN,W,1 To 10 Knots,POINT (-76.74903 37.48865),20801070102,Philbates Creek-York River,51097,28.850,NaN,NaN
2476,YRK031.24,2005-08-02 08:43:00,VADEQ,Overcast (>90%),BAY424,CBP,NaN,NaN,NaN,NaN,...,NaN,WSW,1 To 10 Knots,POINT (-76.79252 37.50465),20801070102,Philbates Creek-York River,51127,29.125,NaN,NaN
2477,ZDM0001,2006-04-26 10:29:00,MDDNR,Scattered To Partly Cloudy (10-50%),BAY437,CBP,NaN,NaN,NaN,NaN,...,NaN,NE,1 To 10 Knots,POINT (-76.50771 38.93598),20600040302,Beards Creek-South River,24003,17.550,NaN,NaN


0       1
1648    1
1649    1
1650    1
1651    1
       ..
828     1
829     1
830     1
823     1
2478    1
Name: coords, Length: 2479, dtype: int64

In [35]:
cbp_cmc["Latitude"].unique().size

2412

In [22]:
station_codes

13136    621010
13138    621110
13139    621130
13140    621160
13141    621170
          ...  
29745    A06773
29746    A06800
29748    A06884
29752    A07086
29757    A07359
Name: USAF, Length: 1538, dtype: object

In [7]:
def raw_noaa_to_dataframe(data, fixed_locs, optional_locs=None):
    columns = [item[0] for item in [*fixed_locs]]
    
    #Adds optional column names if they exist.
    if optional_locs:
        for block in optional_locs:
            columns.extend([item[0] for item in optional_locs[block][1]])

    noaa_list = []
    for line in data:
        var = line[108:].split(b" ")
        fixed_data = parse_fixed_noaa_data(line, fixed_locs)
        optional_data = parse_optional_noaa_data(line, optional_locs)
        data = fixed_data + optional_data
        noaa_list.append(data)
    df = pd.DataFrame(noaa_list, columns = columns, dtype=object)
    fix_noaa_df_dtypes(df, fixed_locs, optional_locs)
    return df

In [16]:
with gzip.open("010020-99999-2019.gz", "rb") as f:
    data = f.read().splitlines()

In [8]:
with gzip.open("data/noaa/2019/A07359-00240-2019.gz", "rb") as f:
    data = f.read().splitlines()

In [12]:
key = ["column_name", "start", "end", "dtype", "nan_value", "conversion factor"]
fixed_locs = [["USAF_ID", 4, 10, "str", None, None], ["NCEI_WBAN_ID", 10, 15, "str", None, None],
              ["Date", 15, 27, "datetime", None, None], ["Data Source", 27, 28, "str", "9", None],
              ["Latitude", 28, 34, "float64", "+99999", 1000], ["Longitude", 34, 41, "float64", "+99999", 1000],
              ["Code", 41, 46, "str", "99999", None], ["Elevation", 46, 51, "int64", "+9999", None],
              ["Call_Letter", 51, 56, "str", "99999", None], ["Quality_Control", 56, 60, "str", "99999", None],
              ["Wind_Dir", 60, 63, "float64", "999", None], ["Wind_Dir_Q", 63, 64, "str", None, None],
              ["Wind_Type", 64, 65, "str", "9", None], ["Wind_Speed", 65, 69, "float64", "9999", 10],
              ["Wind_Speed_Q", 69, 70, "str", None, None], ["Air Temperature", 87, 92, "float64", "+9999", 10],
              ["Air Temperature_Q", 92, 93, "str", None, None], ["Air_Pressure", 99, 104, "float64", "99999", 10],
              ["Air_Pressure_Q", 104, 105, "str", None, None]]

optional_locs = OrderedDict([("AA1",[11, [["Rain_Period1", 3, 5, "float64", "99", None],
                                          ["Rain_Depth1", 5, 9, "float64", "9999", 10],
                                          ["Rain_Condition1", 9, 10, "str", "9", None],
                                          ["Rain_Qual1", 10, 11, "str", None, None]]]),
                             ("AA2",[11, [["Rain_Period2", 3, 5, "float64", "99", None],
                                          ["Rain_Depth2", 5, 9, "float64", "9999", 10],
                                          ["Rain_Condition2", 9, 10, "str", "9", None],
                                          ["Rain_Qual2", 10, 11, "str", None, None]]]),
                             ("AA3",[11, [["Rain_Period3", 3, 5, "float64", "99", None],
                                          ["Rain_Depth3", 5, 9, "float64", "9999", 10],
                                          ["Rain_Condition3", 9, 10, "str", "9", None],
                                          ["Rain_Qual3", 10, 11, "str", None, None]]]),
                             ("AA4",[11, [["Rain_Period4", 3, 5, "float64", "99", None],
                                          ["Rain_Depth4", 5, 9, "float64", "9999", 10],
                                          ["Rain_Condition4", 9, 10, "str", "9", None],
                                          ["Rain_Qual4", 10, 11, "str", None, None]]])])

df = raw_noaa_to_dataframe(data, fixed_locs, optional_locs)
df

,USAF_ID,NCEI_WBAN_ID,Date,Data Source,Latitude,Longitude,Code,Elevation,Call_Letter,Quality_Control,...,Rain_Condition2,Rain_Qual2,Rain_Period3,Rain_Depth3,Rain_Condition3,Rain_Qual3,Rain_Period4,Rain_Depth4,Rain_Condition4,Rain_Qual4
0,A07359,00240,2019-01-01 00:15:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A07359,00240,2019-01-01 00:16:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A07359,00240,2019-01-01 00:35:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A07359,00240,2019-01-01 00:36:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A07359,00240,2019-01-01 00:56:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30215,A07359,00240,2019-12-31 21:56:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30216,A07359,00240,2019-12-31 22:15:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30217,A07359,00240,2020-01-01 04:35:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30218,A07359,00240,2020-01-01 04:36:00,6,42.938,-85.061,FM-15,249,KY70,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
df.dtypes

USAF_ID                      object
NCEI_WBAN_ID                 object
Date                 datetime64[ns]
Data Source                  object
Latitude                    float64
Longitude                   float64
Code                         object
Elevation                     int64
Call_Letter                  object
Quality_Control              object
Wind_Dir                    float64
Wind_Dir_Q                   object
Wind_Type                    object
Wind_Speed                  float64
Wind_Speed_Q                 object
Air Temperature             float64
Air Temperature_Q            object
Air_Pressure                float64
Air_Pressure_Q               object
Rain_Period1                float64
Rain_Depth1                 float64
Rain_Condition1              object
Rain_Qual1                   object
Rain_Period2                float64
Rain_Depth2                 float64
Rain_Condition2              object
Rain_Qual2                   object
dtype: object

In [111]:
optional_locs["AA1"][1]

[['Rain_Period1', 3, 5, 'float64', '99', None],
 ['Rain_Depth1', 5, 9, 'float64', '9999', 10],
 ['Rain_Condition1', 9, 10, 'str', '9', None],
 ['Rain_Qual1', 10, 11, 'str', None, None]]

In [106]:
fixed_locs.extend(optional_locs["AA1"][1])

In [107]:
fixed_locs

[['USAF_ID', 4, 10, 'str', None, None],
 ['NCEI_WBAN_ID', 10, 15, 'str', None, None],
 ['Date', 15, 27, 'datetime', None, None],
 ['Data Source', 27, 28, 'str', '9', None],
 ['Latitude', 28, 34, 'float64', '+99999', 1000],
 ['Longitude', 34, 41, 'float64', '+99999', 1000],
 ['Code', 41, 46, 'str', '99999', None],
 ['Elevation', 46, 51, 'int64', '+9999', None],
 ['Call_Letter', 51, 56, 'str', '99999', None],
 ['Quality_Control', 56, 60, 'str', '99999', None],
 ['Wind_Dir', 60, 63, 'float64', '999', None],
 ['Wind_Dir_Q', 63, 64, 'str', None, None],
 ['Wind_Type', 64, 65, 'str', '9', None],
 ['Wind_Speed', 65, 69, 'float64', '9999', 10],
 ['Wind_Speed_Q', 69, 70, 'str', None, None],
 ['Air Temperature', 87, 92, 'float64', '+9999', 10],
 ['Air Temperature_Q', 92, 93, 'str', None, None],
 ['Air_Pressure', 99, 104, 'float64', '99999', 10],
 ['Air_Pressure_Q', 104, 105, 'str', None, None],
 ['Rain_Period1', 3, 5, 'float64', '99', None],
 ['Rain_Depth1', 5, 9, 'float64', '9999', 10],
 ['Rain_

In [104]:
optional_locs["AA1"][1]

[['Rain_Period1', 3, 5, 'float64', '99', None],
 ['Rain_Depth1', 5, 9, 'float64', '9999', 10],
 ['Rain_Condition1', 9, 10, 'str', '9', None],
 ['Rain_Qual1', 10, 11, 'str', None, None]]

In [85]:
parse_optional_noaa_data(data[0], optional_locs)

[b'01', b'0005', b'9', b'5']
[nan, nan, nan, nan]


In [90]:

    print(x)

['Rain_Period1', 'Rain_Depth1', 'Rain_Condition1', 'Rain_Qual1']
['Rain_Period2', 'Rain_Depth2', 'Rain_Condition2', 'Rain_Qual2']


In [49]:
parse_fixed_noaa_data(ext, optional_locs["AA1"])

[b'01', b'0005', b'9', b'5']

In [ ]:
def 

In [ ]:
for line in data:
    block = extract_noaa_optional_str(line, "A01", 11)
    print(block)

In [26]:
ext

b'AA101000595'

In [ ]:
for line in data:
    term = b"AA1"
    idx = 108 + line[108:].find(term)
    if line.find(term) >= 0:
        print(line[idx:])

In [93]:
data[0]

b'0184A07359002402019010100156+42938-085061FM-15+0249KY70 V0203505N00315000915MN0020125N5+00105+00055999999ADDAA101000595AU120090015AW1415GA1085+000915999GD14991+0009159MA1099865096945REMMET09812/31/18 19:15:01 METAR KY70 010015Z 35006KT 1 1/4SM UP OVC003 01/01 A2949 RMK AO2 P0002 T00100005'